<a href="https://colab.research.google.com/github/thangbomhn87/GEE_Mangrove/blob/main/GEE_GPR_mangrove_traits_and_uncertainties_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPR retrieval of mangrove traits and uncertainties in Google Earth Engine







## Environment setup

In [1]:
!pip install geemap

In [2]:
import geemap
import ee

In [3]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library and set your GEE project for authentication
ee.Initialize(project='ee-ngabinh1987')

## Model import

In [4]:
!git clone -l -s https://github.com/thangbomhn87/GEE_Mangrove_Traits_Uncertainity_Mapping.git
%cd GEE_Mangrove_Traits_Uncertainity_Mapping/GPR_Parameters

import overall_model_gee_LAI
import overall_model_gee_Cab
import overall_model_gee_Cm
import overall_model_gee_Cw

fatal: destination path 'GEE_Mangrove_Traits_Uncertainity_Mapping' already exists and is not an empty directory.
/content/GEE_Mangrove_Traits_Uncertainity_Mapping/GPR_Parameters


In [5]:
# Possible values: 'Cab', 'Cm', 'Cw', 'LAI'
mangroveTrait = 'Cab'
# Possible models: 'overall_model_gee_LAI', 'overall_model_gee_Cab', 'overall_model_gee_Cw', 'overall_model_gee_Cm'
uncModel = overall_model_gee_Cab.models[mangroveTrait]

## Functions

In [6]:
# Function to mask the clouds in S2
def maskS2cloud_and_water(image):
  not_cloud_shadows = image.select('SCL').neq(3);
  not_water = image.select('SCL').neq(6);
  not_cloud_low = image.select('SCL').neq(7);
  not_cloud_medium = image.select('SCL').neq(8);
  not_cloud_high = image.select('SCL').neq(9);
  not_cirrus = image.select('SCL').neq(10);
  not_ice = image.select('SCL').neq(11);

  qa = image.select('QA60');
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).And(not_cloud_shadows)
    .And(not_water).And(not_cloud_low).And(not_cloud_medium).And(not_cloud_high).And(not_cirrus).And(not_ice))
  return image.updateMask(mask).divide(10000).copyProperties(qa).set('system:time_start', qa.get('system:time_start'))

# Auxiliar function for mapping : (1..n) -> (B1..Bn)
def band_names(element):
  bandName = ee.String('B').cat(ee.Number(element).int().format())
  return bandName

# Crop trait GPR mean prediction
def veg_index_GPR(image_orig):
  # Create List of Bands of Dimension n (Xtrain[n,n])
  XTrain_dim = uncModel['X_train'].length().get([0]).getInfo();
  band_sequence   = (ee.List.sequence(1, XTrain_dim).map(band_names));
  # Create a list of band names for flattening operation
  im_norm_ell2D_hypell = image_orig.subtract(ee.Image(uncModel['mx'])).divide(ee.Image(uncModel['sx'])).multiply(ee.Image(uncModel['hyp_ell'])).toArray().toArray(1);
  im_norm_ell2D = image_orig.subtract(ee.Image(uncModel['mx'])).divide(ee.Image(uncModel['sx'])).toArray().toArray(1);
  PtTPt  = im_norm_ell2D_hypell.matrixTranspose().matrixMultiply(im_norm_ell2D).arrayProject([0]).multiply(-0.5);
  PtTDX  = ee.Image(uncModel['X_train']).matrixMultiply(im_norm_ell2D_hypell).arrayProject([0]).arrayFlatten([band_sequence]);
  arg1   = PtTPt.exp().multiply(uncModel['hyp_sig']);
  k_star = PtTDX.subtract(ee.Image(uncModel['XDX_pre_calc']).multiply(0.5)).exp().toArray();
  mean_pred = k_star.arrayDotProduct(ee.Image(uncModel['alpha_coefficients']).toArray()).multiply(arg1);
  mean_pred = mean_pred.toArray(1).arrayProject([0]).arrayFlatten([[uncModel['veg_index']]]);
  mean_pred = mean_pred.add(uncModel['mean_model']);
  image_orig = image_orig.addBands(mean_pred)
  return image_orig.select(uncModel['veg_index'])

# Uncertainty retrieval
def get_GPR_uncertainty(image_orig):
  # Create List of Bands of Dimension n (Xtrain[n,n])
  XTrain_dim = uncModel['X_train'].length().get([0]).getInfo()
  band_sequence   = (ee.List.sequence(1, XTrain_dim).map(band_names))
  # Create a list of band names for flattening operation
  im_norm_ell2D_hypell = image_orig.subtract(ee.Image(uncModel['mx'])).divide(ee.Image(uncModel['sx'])).multiply(ee.Image(uncModel['hyp_ell'])).toArray().toArray(1)
  im_norm_ell2D = image_orig.subtract(ee.Image(uncModel['mx'])).divide(ee.Image(uncModel['sx'])).toArray().toArray(1)
  PtTPt  = im_norm_ell2D_hypell.matrixTranspose().matrixMultiply(im_norm_ell2D).arrayProject([0]).multiply(-0.5)
  PtTDX  = ee.Image(uncModel['X_train']).matrixMultiply(im_norm_ell2D_hypell).arrayProject([0]).arrayFlatten([band_sequence])
  arg1   = PtTPt.exp().multiply(uncModel['hyp_sig']);
  k_star = PtTDX.subtract(ee.Image(uncModel['XDX_pre_calc']).multiply(0.5)).exp().multiply(arg1).toArray()

  variance_vector = ee.Image(uncModel['Linv_pre_calc']).matrixMultiply(k_star.toArray(0).toArray(1)).arrayProject([0])
  variance = ee.Image(uncModel['hyp_sig_unc']).toArray().subtract(variance_vector.arrayDotProduct(variance_vector))
  variance = variance.toArray(1).arrayProject([0]).arrayFlatten([[uncModel['veg_index']+'_unc']])
  image_orig = image_orig.addBands(variance)
  return image_orig.select(uncModel['veg_index']+'_unc').abs().sqrt()

## Area of interest parameters

In [7]:
#Set the Area of Interest.
#The example here focuses on the mangrove forest in Mui Ca Mau National Park Vietnam as a small area to visualize directly in the Colab notebook
#For larger areas, results should be exported to GEE assets or local computers to avoid memory problems

polygon = ee.Geometry.Polygon(
  [[[104.94710610605645,8.605965246171419],
  [104.94710610605645,8.669099010946878],
  [104.76445840097833,8.669099010946878],
  [104.76445840097833,8.605965246171419]]])

studyArea = ee.FeatureCollection('projects/ee-ngabinh1987/assets/DiaPhan_VQG_DatMui')

## Mangrove Vegetation Index (MVI)

In [8]:
# Using mangrove vegetation index (MVI) to mask the non-mangrove pixel
annual_collection  = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  .filterBounds(polygon)
  .filterDate('2019-01-01', '2022-12-31')
  .map(maskS2cloud_and_water)
  .select(['B3', 'B8', 'B11']))

mean_annual_image = annual_collection.median().clip(studyArea);

MVI = mean_annual_image.expression('(b("B8") - b("B3"))/(b("B11") - b("B3"))');

def classify_mangrove(img):
  band = img.select('B8')
  mangrove = band.gt(4.5).rename('mangrove')
  mangrove = mangrove.updateMask(mangrove)
  return img.addBands(mangrove)

MVI_classify = classify_mangrove(MVI);

def clip_StudyArea(image):return image.clip(studyArea)
def clip_MVI(image):return image.updateMask(MVI_classify.select('mangrove'))

## Traits & Uncertainty Estimation

In [10]:
# https://nsidc.org/data/user-resources/help-center/day-year-doy-calendar
Date_Range = ee.Filter.dayOfYear(91, 120);
Year_Range = ee.Filter.calendarRange(2019, 2023,'year');

# Image Collection
img_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  .filterBounds(polygon)
  .filter(Date_Range)
  .filter(Year_Range)
  .map(maskS2cloud_and_water)
  .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])
  .map(clip_StudyArea)
  .map(clip_MVI));

# Predicted mean trait
trait = img_col.map(veg_index_GPR).mean().copyProperties(
  source = img_col.first(),
  properties = ['system:time_start']
  );

# Absolute uncertainty retrieval (SD)
SD = img_col.map(get_GPR_uncertainty).mean().copyProperties(
  source = img_col.first(),
  properties = ['system:time_start']
  );

# Convert to image
imageTrait = ee.Image(trait)
imageSD = ee.Image(SD)

## Visualization

### Visualization parameters

In [11]:
Uncertainty_palette = ['#305FCF', '#919CCC', '#E7E8C3', '#F0BC8B', '#D66C51', '#C44539']
Trait_palette = ['#FFFF80', '#BEF75C', '#38E009', '#36A880', '#225D99', '#0C1078']

vis_params_trait = {'min': 50, 'max': 70, 'palette': Trait_palette}
vis_params_uncertainty = {'min': 5,'max': 20, 'palette': Uncertainty_palette}

### Visualization through geemap

In [12]:
image = ee.Image(imageTrait)

Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(image, vis_params_trait, 'Trait')
Map.add_colorbar(vis_params_trait, label = 'Trait', layer_name = 'Trait', position = 'topright')
Map.centerObject(studyArea, 11)
Map

Map(center=[8.639246468107427, 104.86401340901274], controls=(WidgetControl(options=['position', 'transparent_…

### Visualization uncertainty through geemap

In [13]:
image_unc = ee.Image(imageSD)

Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(image_unc, vis_params_uncertainty, 'uncertainty')
Map.add_colorbar(vis_params_uncertainty, label = 'uncertainty', layer_name = 'uncertainty', position = 'topright')
Map.centerObject(studyArea, 11)
Map

Map(center=[8.639246468107354, 104.86401340901276], controls=(WidgetControl(options=['position', 'transparent_…

## Export images to Google Earth Engine Asset

In [ ]:
# In order to visualize the image in this notebook avoiding memory problems,
# the image should be exported to the GEE asset.

# GEE user name
geeUser = 'ngabinh1987'

# Asset name
assetName_trait = 'Cab_Apr_Mean_2019_2023'
assetName_uncertainty = 'Cab_Apr_Mean_Unc_2019_2023'

assetId_trait = 'users/' + geeUser + '/' + assetName_trait
assetId_uncertainty = 'users/' + geeUser + '/' + assetName_uncertainty

task_config1 = {
    'image': imageTrait,
    'description': 'Cab_Apr_Mean_2019_2023',
    'assetId': assetId_trait,
    'region': studyArea.geometry(),
    'scale': 10,
    'crs' : 'EPSG:4326'
}
task1 = ee.batch.Export.image.toAsset(**task_config1)
task1.start()

task_config2 = {
    'image': imageSD,
    'description': 'Cab_Apr_Mean_Unc_2019_2023',
    'assetId': assetId_uncertainty,
    'region': studyArea.geometry(),
    'scale': 10,
    'crs' : 'EPSG:4326'
}
task2 = ee.batch.Export.image.toAsset(**task_config2)
task2.start()

### Check the status of the export task

In [ ]:
# To visualize the image, the status must be "COMPLETED"
desc1 = task1.status()['description']
state1 = task1.status()['state']

desc2 = task2.status()['description']
state2 = task2.status()['state']

print('The status of the task {} is: {}'.format(desc1,state1))
print('The status of the task {} is: {}'.format(desc2,state2))

The status of the task Cab_Apr_Mean_2019_2023 is: COMPLETED
The status of the task Cab_Apr_Mean_Unc_2019_2023 is: COMPLETED


## References

Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305